In [1]:
import cartopy.crs as ccrs
from datetime import datetime, timedelta,timezone
import gsw
import matplotlib.pyplot as plt
from matplotlib.dates import HourLocator, DateFormatter
import xarray as xr

from ONCToolbox import ONCToolbox
from ONCToolbox.utils import nan_onc_flags, remove_onc_flag_vars
from ONCToolbox.qaqc import location_test,gross_range_test, spike_test, flat_line_test, rate_of_change_test


from ONCToolbox.utils.profilers import split_periods
from ONCToolbox.utils.locations import BCFTerminal

In [2]:
begin_datetime = datetime(2025,7,4)
end_datetime = datetime(2025,7,4,23,59,59,999999)

onct = ONCToolbox()

doi = onct.find_data('BACVP', ['CTD'], begin_datetime, end_datetime)

In [3]:
datasets = []
for loc_code, dev_deps in doi.items():
    for dcc, dep_info in dev_deps.items():
        data = onct.get_data(loc_code, dcc, begin_datetime=begin_datetime,end_datetime=end_datetime)
        datasets.append(data)
ds = xr.combine_by_coords(datasets, combine_attrs='drop_conflicts')
ds = remove_onc_flag_vars(ds) # Remove variables prepended with 'flag_' because we will rerun QAQC tests.

Data quantity is greater than the row limit and will be downloaded in multiple pages.
Estimated approx. 4 pages in total.
Estimated approx. 31 seconds to complete for the rest of the pages.

   (100000 samples) Downloading page 2...
   (200000 samples) Downloading page 3...
   (300000 samples) Downloading page 4...
   (329145 samples) Completed in 35 seconds.


In [4]:
ds = ds.drop_vars(['density','depth','practical_salinity','sigma-t','sigma-theta_0_dbar','sound_speed'], errors = 'ignore')
ds = ds.rename({'conductivity':'sea_water_electrical_conductivity', 'temperature':'sea_water_temperature', 'pressure':'sea_water_pressure'})

## QAQC

Say we are interested in data that from when the VPS is parked at the bottom.
In this example, we will run some QAQC tests. Using the flat line test and the gross range test we can identify when the VPS is at the bottom. Note that the operator range is set to flag pressure data outside of 360-400 dbar.

In [5]:
ranges = {
    'sea_water_electrical_conductivity': {'gross_range': [0, 7], 'operator_range': [2, 6.5]},
    'sea_water_temperature': {'gross_range': [-5, 35], 'operator_range': [1, 30]},
    'sea_water_pressure': {'gross_range': [0, 500], 'operator_range': [390, 400]},
}

for dv in ['sea_water_electrical_conductivity', 'sea_water_temperature', 'sea_water_pressure']:
    ds['flag_gross_range_'+dv] = gross_range_test(ds[dv],sensor_min = ranges[dv]['gross_range'][0], sensor_max = ranges[dv]['gross_range'][1],
                                      operator_min= ranges[dv]['operator_range'][0], operator_max = ranges[dv]['operator_range'][1])
    ds['flag_spike_'+dv] = spike_test(ds[dv])
    ds['flag_flat_line_'+dv] = flat_line_test(ds[dv])

In [6]:
ds = ds.where(ds.flag_gross_range_sea_water_pressure == 1, drop = True) # Remove data outside of operator range.